In [1]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

triplet loss function https://pytorch.org/docs/stable/nn.html#tripletmarginloss

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import pandas as pd

In [4]:
import  datatset
from torch.utils.data import DataLoader

In [5]:
# DataLoader

In [6]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [7]:
label_counts  = df["category_id"].value_counts()

In [8]:
num_calsses = len(label_counts) + 1

In [9]:
labels_ordered = label_counts.keys().tolist()

In [10]:
num_calsses

32094

In [11]:
ds = datatset.HerbTripletDataSet(df_train, TRAIN_PATH,128,label="category_id")

In [12]:
data_loader = DataLoader(ds, batch_size=16, shuffle=True)

# Tracking

In [18]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(TRAIN_PATH/"logs/run-2")

In [19]:
import metrics
from train import Training

In [20]:
acc =  metrics.Accuracy()
f1 = metrics.F1()
manager = metrics.MetricManager([acc, f1], writer)

# Train

In [21]:
class Trainer(Training):
    def __init(self,  metrics, loss, optim, data, epochs, model, save_dir):
        super(self).__init__(self,  metrics, loss, optim, data, epochs, model, save_dir)
        
    def train_step(self, batch):
        pos, neg, anc = batch
        logits = model(pos.img), model(neg.img), model(anc.img) # gotta be a better way
        triplet_loss = triplet_marginloss(*logits)

        # cat logits and labels for ce loss
        join_logits = torch.cat(logits)
        join_lables = torch.cat((pos.label, neg.label, anc.label))

        # Get calssifcation logits
        class_logits = classifier(join_logits)

        ce_loss = cross_entropy(class_logits, join_lables)

        loss = triplet_loss + ce_loss
        if self.metrics.writer:
            self.metrics.writer.add_scalar("loss", loss.item(), self.step)
            self.metrics.writer.add_scalar("cros_entropy_loss", triplet_loss.item(), self.step)
            self.metrics.writer.add_scalar("loss", ce_loss.item(), self.step)

        print(f'loss: {loss}')

        self.optimizer.zero_grad()  # zero gradients
        loss.backward()  # calculate gradients
        self.optimizer.step()  # updated weights



In [22]:
import torch
import torch.nn as nn
import torchvision.models as models

In [23]:
def get_model():
    base = models.resnet18(pretrained=True)
    base.fc  = nn.Linear(in_features=512, out_features=512)
    return base.cuda()

In [24]:
classifier = nn.Linear(in_features=512, out_features=32204).cuda()

In [25]:
model = get_model()

In [26]:
triplet_marginloss = nn.TripletMarginLoss()
cross_entropy = nn.CrossEntropyLoss()

In [28]:
layer_lrs = [
    {'params': model.layer1.parameters(), 'lr': 1e-6},
    {'params': model.layer2.parameters(), 'lr': 1e-6},
    {'params': model.layer3.parameters(), 'lr': 1e-5},
    {'params': model.layer4.parameters(), 'lr': 1e-4},
    {'params': model.fc.parameters(), 'lr': 1e-3},
    {'params': classifier.parameters(), 'lr': 1e-3}
]

In [30]:
optimizer = torch.optim.Adam(layer_lrs)

In [31]:
model_path=TRAIN_PATH/"models/"

In [32]:
! export CUDA_LAUNCH_BLOCKING=1

In [36]:
Trainer(manager, None, optimizer, data_loader, 3, None, model_path).run()

Epoch 0/3


[autoreload of train failed: Traceback (most recent call last):
  File "/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 317, in update_instances
    update_instances(old, new, obj, visited)
  File "/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/IPython/extensions/autoreload.py", lin

loss: 11.5514554977417
loss: 11.519807815551758
loss: 11.51269245147705
loss: 11.379258155822754
loss: 11.372251510620117
loss: 11.698086738586426
loss: 11.748482704162598
loss: 11.663235664367676
loss: 12.047969818115234
loss: 11.81884479522705
loss: 11.641877174377441
loss: 11.367223739624023
loss: 11.328934669494629
loss: 12.128104209899902
loss: 12.123634338378906
loss: 11.382646560668945
loss: 11.89951229095459
loss: 11.474444389343262
loss: 11.532544136047363
loss: 11.695939064025879
loss: 11.4854154586792
loss: 11.543900489807129
loss: 11.357575416564941
loss: 11.493578910827637
loss: 11.530760765075684
loss: 11.655909538269043
loss: 11.53950309753418
loss: 11.565032958984375
loss: 11.347163200378418
loss: 11.613289833068848
loss: 11.855154991149902
loss: 11.50229263305664
loss: 11.94113826751709
loss: 11.851414680480957
loss: 11.27842903137207
loss: 11.825401306152344
loss: 11.853523254394531
loss: 11.184727668762207
loss: 11.420722007751465
loss: 11.655486106872559
loss: 12.39

AttributeError: 'NoneType' object has no attribute 'state_dict'